# [NAPARI](https://napari.org/)
## attempting to make color-coded movies for calcium imaging data

In [ ]:
import napari
from magicgui import magicgui
from enum import Enum
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import skimage
from skimage.io import imread
from skimage.measure import regionprops_table
import napari_nikon_nd2
import os 
from magicgui import magicgui
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
import ipywidgets
from tqdm import tqdm
import time
import sys
from plotParams import *


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

> using unix '!ls' command; do not have a Windows machine to test replacement

In [ ]:
def loadMovie(dirPath,fString):
    fList = !ls $dPath/*$gene*.tif
    movie = np.asarray([imread(f) for f in fList])
    # sciscan saves 12-bit images into 16-bit data files in the midrange
    # so there should be 2^12 = 4096 levels of encoded signal and a lot of empty data
    # but only half of the 12-bit range is used (positive values???) so that data is only 2048 levels?
    # Empirically found that this works, but I don't understand exactly how the data is encoded
    movie[movie==pow(2,16)]=0 # seems like the last bit of the 16-bit image is used to encode something else
    movie = np.subtract(movie,pow(2,15)) # this should bring the data to span from 0 to 4096
    movie[movie>=65000]=0 #HACK!!! There's some lingering data in final bits of image that don't belong anywhere
    # really need to dig into what sci-scan does
    return movie

def removeLayer(viewer,lName):
    lNames = []
    for l in viewer.layers:
        lNames.append(l.name)
    if lName in lNames:
        viewer.layers.remove(viewer.layers[lName])

In [ ]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

In [ ]:
# viewer.close()

***
# Index <a id='Index'></a>
***
- [Baseline definition](#F0Extract)

***
## Load movies to try to identify baseline<a id='F0Extract' name='F0Extract'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# path to directory that holds tif image sequence
dPath = "/Users/angueyraaristjm/Documents/eelImaging/2Pdata/20220615_5dpf_L02_01/"
fString = 'vamp2'
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()
movie = loadMovie(dPath,fString)
viewer.add_image(movie, contrast_limits=[0,2048])


In [ ]:
tlims = [1,4]; # pick frames to be used for baseline (the 'F0')
# removeLayer(viewer,'mean')
l = viewer.layers['movie']
# calculate mean of baseline and add to napari
mean = l.data[tlims[0]:tlims[1]].mean(axis=0)
viewer.add_image(mean, blending='additive', contrast_limits=[0,2048], colormap = l.colormap, name = "mean")

# subtract baseline from entire movie (the 'deltaF')
# removeLayer(viewer,'dF')
dF = np.empty(movie.shape)
for t in range(movie.shape[0]):
    dF[t,:,:] = movie[t,:,:]-mean
    
viewer.add_image(dF, blending='additive', contrast_limits=[-2048,2048], colormap = 'twilight', name = "dF")

In [ ]:
# Qiuxiang's paper recommends binning to smooth out.
# Testing binning strategies to see if it's worth it
# for time binning adjust first parameter of block_size=(time_bin, 1, 1)
# the second and third parameters adjust binning in x and y block_size=(time_bin, x, y)
viewer.layers.remove(viewer.layers['dF_binned'])
dF_binned = skimage.measure.block_reduce(dF, block_size=(2, 1, 1), func=np.mean)
viewer.add_image(dF_binned, blending='additive', colormap = 'twilight', name = "dF_binned")